In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

import networkx as nx
from node2vec import Node2Vec as n2v
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity

from utils import get_test_edges, get_graph, get_links, recall_at_k

## Data preparation, train-test split

In [2]:
data_tr = pd.read_csv("vk_friends_test_candidate/data_tr.csv")
full_data = data_tr.drop('t', axis=1)
full_data['h'] = full_data['h'] + 1
test_edges = get_test_edges(data_tr)
full_graph = get_graph(full_data)
train_graph = full_graph.copy()
train_graph.remove_edges_from(test_edges.values)

In [3]:
print("Test edges ratio:", test_edges.shape[0] / len(full_graph.edges()))

Test edges ratio: 0.19512847415087742


## Training a node2vec embeddings

In [4]:
g_emb = n2v(train_graph, dimensions=64, walk_length=80, p=1, q=1, num_walks=50, workers=4)

WINDOW = 2 
MIN_COUNT = 2
BATCH_WORDS = 4

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS,
)

# create embeddings dataframe
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in train_graph.nodes()],
        index = train_graph.nodes
    )
)

Computing transition probabilities:   0%|          | 0/13489 [00:00<?, ?it/s]

## Making predictions

In [5]:
def predict_links(G, df, user_id, N):
    """This function will predict the top N links a node (user_id) should be connected with
        which it is not already connected with in G.
    
    Args:
        G (netowrkx.Graph) : The network used to create the embeddings
        df (pd.DataFrame) : The dataframe which has embeddings associated to each node
        user_id (int) : The user you're interested 
        N (int) : The number of recommended links you want to return
        
    Returns:
        A list of nodes the input node should be connected with.
    """
    
    # separate target user with all others
    user = df[df.index == user_id]
    
    # other users are all users which the current doesn't have an edge connecting
    all_nodes = G.nodes()
    other_nodes = [n for n in all_nodes if n not in list(G.adj[user_id]) + [user_id]]
    other_users = df[df.index.isin(other_nodes)]
    # get similarity of current user and all other users
    sim = cosine_similarity(user, other_users)[0].tolist()
    
    idx = other_users.index.tolist()
    
    # create a similarity dictionary for this user w.r.t all other users
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)
    similar_users = idx_sim[1:N+1]
    users = [art[0] for art in similar_users]
    return users

In [6]:
dict_actual = get_links(test_edges)

In [7]:
test_users = test_edges.u.unique()
dict_predicted = {user: set(predict_links(train_graph, emb_df, user, 10)) for user in  tqdm(test_users)}

100%|██████████████████████████████████████████████████████████████████████████████| 1897/1897 [01:17<00:00, 24.34it/s]


In [8]:
recall_at_k(dict_predicted, dict_actual)

0.10648392198207696

## Writing to a file

In [9]:
all_users = list(full_graph.nodes()) 
dict_predicted_all = {user: set(predict_links(train_graph, emb_df, user, 10)) for user in  tqdm(all_users)}

100%|████████████████████████████████████████████████████████████████████████████| 13489/13489 [07:21<00:00, 30.54it/s]


In [10]:
with open('recommendation_1_link_pred.txt', 'w') as file:
    for key, value in dict_predicted_all.items(): 
         file.write('%s: %s\n' % (key,  ", ".join(map(str,value))))

## Creating training/testing data for gradient boosting

In [11]:
def random_pairs(number_list): 
    """This is an auxiliary function that generates random pairs of number from the given list."""
    return [number_list[i] for i in random.sample(range(len(number_list)), 2)]

In [12]:
unique_nodes = list(train_graph.nodes())
pairs = {tuple(random_pairs(unique_nodes)) for i in range(39000)}
pairs = pairs - set(train_graph.edges())
non_edges_df = pd.DataFrame(pairs, columns=['u', 'v'])
all_possible_edges = set(train_graph.edges()) | pairs

In [13]:
# generate edge features for all pairs of nodes
edge_features = [
    (mdl.wv.get_vector(str(i)) + mdl.wv.get_vector(str(j))) for i,j in all_possible_edges
]

# get current edges in the network
edges = list(train_graph.edges())

# create target list, 1 if the pair exists in the network, 0 otherwise
is_con = [1 if e in edges else 0 for e in all_possible_edges]

print(sum(is_con))

39124


In [14]:
# get training and target data
X = np.array(edge_features)
y = is_con

# train test split
x_train, x_test, y_train, y_test = train_test_split(
  X,
  y,
  test_size = 0.3
)

## Training and evaluation

In [15]:
# GBC classifier
clf = CatBoostClassifier(iterations=100, depth=10, random_state=0, verbose=0)

# train the model
clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)
y_true = y_test

y_pred = clf.predict(x_test)
x_pred = clf.predict(x_train)
test_acc = accuracy_score(y_test, y_pred)
train_acc = accuracy_score(y_train, x_pred)
print("Testing Accuracy : ", test_acc)
print("Training Accuracy : ", train_acc)
print("roc_auc:", roc_auc_score(y_test, y_pred))

print("Test Confusion Matrix : ")
print(confusion_matrix(y_pred,y_test))

print("Test Classification Report : ")
print(classification_report(y_test, clf.predict(x_test)))

Testing Accuracy :  0.9502005632841171
Training Accuracy :  0.99654339953912
roc_auc: 0.9500527530491852
Test Confusion Matrix : 
[[10843   401]
 [  766 11424]]
Test Classification Report : 
              precision    recall  f1-score   support

           0       0.96      0.93      0.95     11609
           1       0.94      0.97      0.95     11825

    accuracy                           0.95     23434
   macro avg       0.95      0.95      0.95     23434
weighted avg       0.95      0.95      0.95     23434



## Forming recommendations

In [16]:
def predict_ranked(user_id, predicted):
    confidence = []
    for pred_user in predicted:
        pred_ft = [(mdl.wv.get_vector(str(user_id))+mdl.wv.get_vector(str(pred_user)))]
        confidence.append(clf.predict_proba(pred_ft)[0][1])
    confidence = np.array(confidence)
    predicted = np.array(predicted)
    return list(predicted[confidence.argsort()[::-1][:10]])

In [17]:
dict_predicted_ranked = {}
for user in test_users:
    predicted = predict_links(train_graph, emb_df, user, 50)
    dict_predicted_ranked[user] = predict_ranked(user, predicted)

In [18]:
user_ids = pd.read_csv("vk_friends_test_candidate/user_ids.csv")

In [19]:
users_in_graph = set(user_ids.u.values) & set(all_users)
new_users = set(user_ids.u.values) - set(all_users)

In [20]:
dict_predicted_ranked = {}
for user in users_in_graph:
    predicted = predict_links(train_graph, emb_df, user, 50)
    dict_predicted_ranked[user] = predict_ranked(user, predicted)

In [21]:
the_most_popular_users = list(full_data.u.value_counts()[:10].index)
for user in new_users:
    dict_predicted_ranked[user] = the_most_popular_users

## Writing to a file

In [22]:
with open('recommendation_2_link_pred.txt', 'w') as file:
    for key, value in dict_predicted_ranked.items(): 
         file.write('%s: %s\n' % (key,  ", ".join(map(str,value))))